In [1]:
import json
import pandas as pd

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/less30_acl25_gpt4/without_module_head/conversations.json'
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/less30_acl25_gpt4/without_module_head/graph.jsonl'

In [3]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

# Example usage
input_text = """
some initial text
module name="example1" \n   attribu\nte="   value";
some other text
module name="example2" attribute="another value";
"""

# Find the first occurrence
result = find_module_string(input_text)
print("Found substring:", result)

# If you need to find all occurrences instead of just the first one, you can use re.findall
all_matches = re.findall(r"module.*?;", input_text, re.DOTALL)
print("All found substrings:", all_matches)

Found substring: module name="example1" 
   attribu
te="   value";
All found substrings: ['module name="example1" \n   attribu\nte="   value";', 'module name="example2" attribute="another value";']


In [4]:
graph = pd.read_json(graph_path, lines=True)
with open(conv_path, 'r') as f:
    conv = json.load(f)


In [5]:
conv[0]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'CLK_in', 'type': 'input port'}, {'id': 1, 'content': 'RST', 'type': 'input port'}, {'id': 2, 'content': 'CLK_15', 'type': 'output port'}, {'id': 3, 'content': 'CLK_3', 'type': 'output port'}, {'id': 4, 'content': 'div_15', 'type': 'submodule'}, {'id': 5, 'content': 'div_3', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[0, 1, 0, 1, 4, 5], [4, 4, 5, 5, 2, 3]]\n, Design a hierarchical Verilog module for a frequency divider that accepts a 60MHz input clock and outputs two different clock signals: 15MHz and 3MHz.\n\nModule name:\n    multi_freq_div\n\nInput ports:\n    CLK_in: Input clock signal\n    RST: Reset signal\n\nOutput ports:\n    CLK_15: Output clock signal with a frequency of CLK_in divided by 4.\n    CLK_3: Output clock signal with a frequency of CLK_in divided by 20.\n\nImplementation:\nThe module uses two submodules:\n- clk_div_4 to generate the 15MHz clock by counting to 1 and

In [6]:
conv[0]['conversations'][1]['value']

'module multi_freq_div (\n    input CLK_in,\n    input RST,\n    output CLK_15,\n    output CLK_3\n);\n\n    wire clk_15_out;\n    wire clk_3_out;\n\n    clk_div_4 div_15 (\n        .CLK_in(CLK_in),\n        .RST(RST),\n        .CLK_out(clk_15_out)\n    );\n\n    clk_div_20 div_3 (\n        .CLK_in(CLK_in),\n        .RST(RST),\n        .CLK_out(clk_3_out)\n    );\n\n    assign CLK_15 = clk_15_out;\n    assign CLK_3 = clk_3_out;\n\nendmodule\n\nmodule clk_div_4 (\n    input CLK_in,\n    input RST,\n    output reg CLK_out\n);\n    reg cnt;\n\n    always @(posedge CLK_in or posedge RST) begin\n        if (RST) begin\n            CLK_out <= 0;\n            cnt <= 0;\n        end else if (cnt == 1) begin\n            CLK_out <= ~CLK_out;\n            cnt <= 0;\n        end else begin\n            cnt <= cnt + 1;\n        end\n    end\nendmodule\n\nmodule clk_div_20 (\n    input CLK_in,\n    input RST,\n    output reg CLK_out\n);\n    reg [4:0] cnt;\n\n    always @(posedge CLK_in or posedge 

In [7]:
head = find_module_string(conv[0]['conversations'][1]['value'])
conv[0]['conversations'][0]['value'] + head

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'CLK_in', 'type': 'input port'}, {'id': 1, 'content': 'RST', 'type': 'input port'}, {'id': 2, 'content': 'CLK_15', 'type': 'output port'}, {'id': 3, 'content': 'CLK_3', 'type': 'output port'}, {'id': 4, 'content': 'div_15', 'type': 'submodule'}, {'id': 5, 'content': 'div_3', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[0, 1, 0, 1, 4, 5], [4, 4, 5, 5, 2, 3]]\n, Design a hierarchical Verilog module for a frequency divider that accepts a 60MHz input clock and outputs two different clock signals: 15MHz and 3MHz.\n\nModule name:\n    multi_freq_div\n\nInput ports:\n    CLK_in: Input clock signal\n    RST: Reset signal\n\nOutput ports:\n    CLK_15: Output clock signal with a frequency of CLK_in divided by 4.\n    CLK_3: Output clock signal with a frequency of CLK_in divided by 20.\n\nImplementation:\nThe module uses two submodules:\n- clk_div_4 to generate the 15MHz clock by counting to 1 and

In [8]:
for i in range(len(conv)):
    module_head = find_module_string(conv[i]['conversations'][1]['value'])
    conv[i]['conversations'][0]['value'] = conv[i]['conversations'][0]['value'] + '\n' + module_head

In [9]:
conv[100]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk', 'type': 'input port'}, {'id': 1, 'content': 'rst_n', 'type': 'input port'}, {'id': 2, 'content': 'enable', 'type': 'input port'}, {'id': 3, 'content': 'data_in', 'type': 'input port'}, {'id': 4, 'content': 'data_out', 'type': 'output port'}, {'id': 5, 'content': 'saturation_flag', 'type': 'output port'}, {'id': 6, 'content': 'u_accumulator', 'type': 'submodule'}, {'id': 7, 'content': 'u_control', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[0, 3, 2, 1, 0, 6, 1, 7, 7], [6, 6, 6, 6, 7, 7, 7, 4, 5]]\n, Design a modular Verilog implementation for a 16-bit wide arithmetic accumulator with a saturating feature. The accumulator should accept 16-bit signed integers, sum them continuously, and saturate the output to the maximum or minimum 16-bit signed value on overflow or underflow respectively. Include a signal to indicate when saturation occurs. The accumulator should reset to zero on 

In [10]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/less30_acl25_gpt4/with_module_head/conversations.json'



In [11]:
with open(new_conv_path, 'w') as f:
    json.dump(conv, f)

In [12]:
import json
with open(new_conv_path, 'r') as f:
    new_conv = json.load(f)
    


In [13]:
len(new_conv)

480

In [ ]:
new_conv[100]['conversations'][0]['value']

In [32]:
len(graph)

2148